In [1]:
!pip install faiss-cpu==1.8.0 -q
!pip install langchain==0.2.5 -q
!pip install langchain-community==0.2.5 -q
!pip install langchain-core===0.2.7 -q
!pip install langchain-openai==0.1.15 -q
!pip install langchain-text-splitters==0.2.1 -q
!pip install langsmith==0.1.81 -q
!pip install openai==1.10.0 -q
!pip install tiktoken==0.7.0 -q
!pip install pdfminer.six -q
!pip install langchain-google-genai -q
!pip install pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.1/366.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-text-splitters 0.2.2 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.2.7 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import sys
import os
import numpy as np
import pandas as pd
import time
import pickle
from langchain_community.document_loaders import PyPDFLoader, PDFMinerLoader
from langchain_community.vectorstores import FAISS

from langchain.chains import  ConversationalRetrievalChain ,RetrievalQA
from langchain_community.llms import OpenAI
from langchain_community.embeddings import OpenAIEmbeddings

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate
from langchain.schema import HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from google.colab import userdata
from langchain.chains.question_answering import load_qa_chain
from sklearn.metrics.pairwise import cosine_similarity
#import warnings
#warnings.filterwarnings("ignore")

In [4]:
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [15]:
cumle_1 = "Üretken yapay zeka geleceği değiştirecek."
cumle_2 = "Futbol izlemeyi severim."

In [16]:
embeddings =  GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

word1_embedding = embeddings.embed_query(cumle_1)
word2_embedding = embeddings.embed_query(cumle_2)

In [19]:
word1_embedding

[0.026272809132933617,
 0.013692772015929222,
 -0.001989892451092601,
 -0.08925680816173553,
 -0.00556972436606884,
 0.03492771461606026,
 0.03527803346514702,
 0.01382189616560936,
 0.015496239997446537,
 0.01937832310795784,
 0.03645262122154236,
 0.07134585082530975,
 0.05388927832245827,
 -0.0382661335170269,
 -0.018276341259479523,
 -0.0074222879484295845,
 -0.03708923980593681,
 0.003902883268892765,
 -0.0771062970161438,
 -0.01289051678031683,
 0.03679140284657478,
 -0.018922068178653717,
 -0.011161916889250278,
 0.02839452214539051,
 -0.07842545211315155,
 -0.027544284239411354,
 0.009023779071867466,
 -0.026922961696982384,
 -0.0447220616042614,
 0.005962049122899771,
 0.010461724363267422,
 0.0007376025896519423,
 0.04036587104201317,
 -0.04070056229829788,
 0.0665535032749176,
 -0.018016479909420013,
 -0.04938032105565071,
 0.0307001993060112,
 -0.003422304755076766,
 -0.02737794630229473,
 -0.045215897262096405,
 0.026383979246020317,
 -0.010116680525243282,
 0.013632330112

In [17]:
len(word1_embedding) , len(word2_embedding)

(768, 768)

In [18]:
similarity = cosine_similarity(np.array(word1_embedding).reshape(1, -1), np.array(word2_embedding).reshape(1, -1))[0][0]
print(similarity)

0.5599950357613743
